In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
from datetime import datetime

In [2]:
class ZacksSpider(scrapy.Spider):
    name = "Zacks"
    allowed_domains = ["zacks.com"]

    def start_requests(self):
        tickers = json.loads(open('./tickers.json', encoding='utf-8').read())
        for ticker in tickers:
            yield scrapy.Request('https://www.zacks.com/stock/quote/' + ticker['Symbol'],
                                 meta={'ticker':ticker['Symbol']})

    def parse(self, response):
        anchor = response.css('.spl_sup_text+ a::text').extract_first()
        if anchor is not None:
            date_str_array = anchor.split('/')
            yield {
                'ticker': response.meta['ticker'],
                'nextReportDate': datetime(int('20' + date_str_array[2]),
                                           int(date_str_array[0]),
                                           int(date_str_array[1]))
            }

In [3]:
process = CrawlerProcess({
    'CONCURRENT_REQUESTS' : 5,
    'DOWNLOAD_DELAY' : 1
})

process.crawl(ZacksSpider)
process.start()

2017-03-03 12:06:30 [scrapy.utils.log] INFO: Scrapy 1.3.2 started (bot: scrapybot)
2017-03-03 12:06:30 [scrapy.utils.log] INFO: Overridden settings: {'CONCURRENT_REQUESTS': 5, 'DOWNLOAD_DELAY': 1}
2017-03-03 12:06:30 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2017-03-03 12:06:30 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrap